In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns

# 데이터 파악 및 전처리

In [ ]:
dataframe = pd.read_csv('../input/my-dataset/credit_train.csv')
dataframe

In [ ]:
for i in range(100000 ,100514):  #하단에 붙어있는 결측값 제거
  dataframe = dataframe.drop([i])

dataframe

* 인적사항
 - index
 - Customer ID


* 대출 관련 정보 
 - Loan ID
 - Term 기간
 - Purpose 목적 
 - Current Loan Amount 상환액
 - Monthly Debt 월간 납입액 
 - Maximum Open Credit OpenCredit 한도 (마이너스통장)
 - Number of Open Accounts 계좌 수 
 
 
* 신용리스크 관련 정보
 - Credit Score 신용평가점수 
 - Years of Credit History 신용정보 년수 (최소 7년이 있어야 좋은상태가 된다)
 - Months since last delinquent 마지막 연체기록
 - Number of Credit Problems 
 - Bankruptcies 파산횟수
 - Tax Liens 선취특권, 담보물권, 유치권


* 소득정보
 - Annual Income 연간 소득
 - Years in current job 근속연수
 - Home Ownership 집 소유방식
 - Current Credit Balance 대변잔액


In [ ]:
np.around(dataframe.describe(),1)

In [ ]:
#결측값치 확인
dataframe[list(dataframe.columns)].isnull().sum()

# 결측치를 포함하는 컬럼 확인

In [ ]:
#Tax_lien
print(dataframe['Tax Liens'].value_counts())
print(dataframe['Tax Liens'].unique())
print(dataframe['Tax Liens'].isnull().sum())

In [ ]:
#Tax_Lien에서 결측치는 최빈값인 0으로 가정
dataframe['Tax Liens'] = dataframe['Tax Liens'].fillna(0)

In [ ]:
#Bankruptcies
print(dataframe['Bankruptcies'].value_counts())
print(dataframe['Bankruptcies'].unique())
print(dataframe['Bankruptcies'].isnull().sum())

In [ ]:
#Bankruptcies에서 결측치는 최빈값인 0으로 가정
dataframe['Bankruptcies'] = dataframe['Bankruptcies'].fillna(0)

Years in current job
두 가지 분류
1. 소득을 올릴 수 있는 직업이 없는 경우(무직자)_ Annual Income & Years in current job ==nan
2. 직장 특성 상 근속연수를 기입할 수 없는 경우(eg.프리랜서)_ Years in current job ==nan but Annual Income !=nan


In [ ]:
#Case No.1: 'Years in current job'이 NaN인 사람 중에서 'Annual Income' 역시 NaN인 경우
dataframe[dataframe['Years in current job'].isnull()]['Annual Income'].isnull().sum()

In [ ]:
#Case No.1인 경우 810 -> 이 경우에는 'Years in current job'을 'None'으로 채운다.
dataframe.loc[dataframe['Annual Income'].isnull(), 'Years in current job'] = 'None'


In [ ]:
#Case No.2: 'Years in current job'이 NaN이지만 "Annual Income"은 있는 경우
dataframe.loc[dataframe['Years in current job'].isnull(),'Years in current job'] = 'casual job'

In [ ]:
#Months since last delinquent | NaN이 가장 높은 수치고, 그다음 숫자가 높을 수록 좋은 것
#신용관련 정보가 20년까지 저장된다고 가정 -> 모든 값에 -12*20을 해서 NaN인 사람을 0으로 수렴
dataframe.loc[dataframe['Months since last delinquent'].isnull(),'Months since last delinquent'] = 240
dataframe['Months since last delinquent'] = dataframe['Months since last delinquent'] - 240

In [ ]:
#숫자가 똑같은 Annual Income과 Credit Score 결측치에 대한 검증
NaNcount_AnnualIncome = []
NaNcount_CreditScore = []
NaNcount = []
for i in range(len(dataframe)):
    if np.isnan(dataframe['Annual Income'][i]) == True:
        NaNcount_AnnualIncome += [i]

for i in range(len(dataframe)):
    if np.isnan(dataframe['Credit Score'][i]) == True:
        NaNcount_CreditScore += [i]
        
    
for i in range(len(NaNcount_AnnualIncome)):
    if NaNcount_AnnualIncome[i] == NaNcount_CreditScore[i]:
        NaNcount += [i]

        
print('NaNcount_AnnualIncome의 위치정보리스트 길이:',len(NaNcount_AnnualIncome))
print('NaNcount_CreditScore의 위치정보리스트 길이:',len(NaNcount_CreditScore))
print('NaNcount의 위치정보리스트 길이:',len(NaNcount))

Credit Score가 NaN이면 Annual income도 NaN임을 알 수 있다.
따라서 아직까지 경제활동에 대한 소득이 발생하지 않아 Credit Score가 부여되지 않았다고 가정하고, 결측치는 모두 0으로 채운다. 

In [ ]:
dataframe.loc[dataframe['Credit Score'].isnull(),'Credit Score'] = 0
dataframe.loc[dataframe['Annual Income'].isnull(),'Annual Income'] = 0

In [ ]:
#수정된 데이터의 결측치 확인
dataframe[list(dataframe.columns)].isnull().sum()

In [ ]:
np.around(dataframe.describe(),1)

In [ ]:
dataframe['Maximum Open Credit'].describe().astype('int')

데이터의 분산도 크고 평균과 중앙값에서 큰 차이를 보여 결측값을 채울 대표값 선정이 어렵다. 또한 결측치도 2개 뿐이라 별도의 값으로 채워넣기 보단, 제거한다. 

In [ ]:
dataframe = dataframe.dropna()
print(dataframe.isnull().sum())
print(len(dataframe))

## 데이터 이상치 확인

 - Current Loan Amount: 최대값 99999999.0
 - Credit Score: 최대값 7510.0 (원래 범위 300~850)

In [ ]:
#Credit Score 원래 범위는 300~ 850 그중에서 일반적으로 700점이 넘으면 좋다고 판단
#이상치 확인을 위한 시각화 (최대치인 850이 넘은 레코드 대상)
sns.distplot(dataframe.loc[dataframe['Credit Score']>850, ['Credit Score']])

In [ ]:
#주요 점수대인 700~750 사이가 가장많이 분포하는 것으로보아 '0'이 하나 더 붙어있음을 유추
dataframe.loc[dataframe['Credit Score']>850, ['Credit Score']] = (dataframe.loc[dataframe['Credit Score']>850, ['Credit Score']])/10

In [ ]:
#Current Loan amount 데이터 이상치 확인을 위한 시각화
sns.distplot(np.around(dataframe['Current Loan Amount'], 1))

In [ ]:
# Current Loan Amount의 Max 값인 99999999가 이상치임을 확인
temp = dataframe.loc[dataframe['Current Loan Amount']==99999999, ['Current Loan Amount', 'Term', 'Purpose', 'Loan Status']]
print(temp['Purpose'].value_counts())
print('-'*20)
print(temp['Loan Status'].value_counts())
print('-'*20)
print(temp['Term'].value_counts())

 - 목적이나 빌린 기간이 다양함에도 불구하고 전원 Fully paid된 것을 보아, Current Loan Amount 데이터가 유실 및 손상된 것이라고 가정
 - 이를 추청하기 위해서 LinearRgression을 이용해, 회귀분석을 해보았지만 타당한 값을 낼 수 없어 해당 데이터들을 배제하고 진행

In [ ]:
dataframe = dataframe.loc[dataframe['Current Loan Amount']!=99999999]
dataframe

# 데이터별 분포확인

In [ ]:
Continuous = ['Loan Status', 'Current Loan Amount', 'Credit Score', 'Annual Income', 'Monthly Debt', 'Years of Credit History', 'Months since last delinquent', 'Number of Open Accounts', 'Number of Credit Problems', 'Current Credit Balance', 'Maximum Open Credit', 'Bankruptcies', 'Tax Liens']
Category = ['Loan Status', 'Term', 'Years in current job', 'Home Ownership', 'Purpose']


In [ ]:
sns.pairplot(dataframe[Continuous], hue='Loan Status')

In [ ]:
plt.figure(figsize=(13,13))
for i in range(1, 11):
  plt.subplot(5,2,i)
  sns.distplot(dataframe.loc[dataframe['Loan Status']=='Fully Paid', Continuous[i]], label='Fully Paid')
  sns.distplot(dataframe.loc[dataframe['Loan Status']=='Charged Off', Continuous[i]], label= 'Charged Off')
  plt.legend(loc = 'best')
plt.show()

In [ ]:
plt.figure(figsize=(15 ,15))
for i in range(1,7):
  plt.subplot(2,3,i)
  sns.countplot(x=Category[i], data =dataframe, hue='Loan Status')
plt.show()

* # 머신러닝을 위한 더미화 및 정규화

In [ ]:
Continuous.remove('Loan Status')
Category.remove('Loan Status')

In [ ]:
#연속형 데이터들 MinMax 방식으로 정규화 진행
from sklearn.preprocessing import MinMaxScaler
scale = MinMaxScaler().fit(dataframe[Continuous])
scaled_dataframe = pd.DataFrame(scale.transform(dataframe[Continuous]), columns= Continuous, index= dataframe.index)

In [ ]:
#범주형 데이터들 더미화 진행
df_dummies = pd.get_dummies(dataframe[Category], drop_first= True)
df_dummies

In [ ]:
findataframe = pd.concat([scaled_dataframe, df_dummies], axis= 1)
findataframe

# 데이터셋 분리

In [ ]:
temp = list(findataframe.columns)
temp.sort()
temp

In [ ]:
loanlist = ['Term_Short Term',
            'Current Loan Amount',
            'Maximum Open Credit',
            'Number of Open Accounts',
            'Purpose_Buy House',
            'Purpose_Buy a Car',
            'Purpose_Debt Consolidation',
            'Purpose_Educational Expenses',
            'Purpose_Home Improvements',
            'Purpose_Medical Bills',
            'Purpose_Other',
            'Purpose_Take a Trip',
            'Purpose_major_purchase',
            'Purpose_moving',
            'Purpose_other',
            'Purpose_renewable_energy',
            'Purpose_small_business',
            'Purpose_vacation',
            'Purpose_wedding',]

creditlist = ['Credit Score',
              'Years of Credit History',
              'Months since last delinquent',
              'Number of Credit Problems',
              'Bankruptcies',
              'Tax Liens']

incomelist = ['Annual Income',
              'Current Credit Balance',
              'Years in current job_10+ years',
              'Years in current job_2 years',
              'Years in current job_3 years',
              'Years in current job_4 years',
              'Years in current job_5 years',
              'Years in current job_6 years',
              'Years in current job_7 years',
              'Years in current job_8 years',
              'Years in current job_9 years',
              'Years in current job_< 1 year',
              'Years in current job_None',
              'Years in current job_casual job',
              'Home Ownership_Home Mortgage',
              'Home Ownership_Own Home',
              'Home Ownership_Rent']

In [ ]:
#dataset 분리
dataset_loan = findataframe[loanlist]
dataset_credit = findataframe[creditlist]
dataset_income = findataframe[incomelist]
#Target 설정
target = dataframe['Loan Status']
target = target.replace('Fully Paid', 1)
target = target.replace('Charged Off', 0)

#train, test set 분리
from sklearn.model_selection import train_test_split
xtrain_loan, xtest_loan, ytrain, ytest = train_test_split(dataset_loan, target, test_size=0.2, stratify = target)

#같은 레코드를 분리
train_indexlist = list(xtrain_loan.index)
test_indexlist = list(xtest_loan.index)
xtrain_credit = dataset_credit.loc[train_indexlist]
xtest_credit = dataset_credit.loc[test_indexlist]
xtrain_income = dataset_income.loc[train_indexlist]
xtest_income = dataset_income.loc[test_indexlist]


## 머신러닝_RogisticRegression

In [ ]:
from sklearn.linear_model import LogisticRegression
log_loan = LogisticRegression().fit(xtrain_loan, ytrain)
log_credit = LogisticRegression().fit(xtrain_credit, ytrain)
log_income = LogisticRegression().fit(xtrain_income, ytrain)

In [ ]:
print(log_loan.score(xtrain_loan, ytrain))
print(log_loan.score(xtest_loan, ytest))
print('-'*20)
print(log_credit.score(xtrain_credit, ytrain))
print(log_credit.score(xtest_credit, ytest))
print('-'*20)
print(log_income.score(xtrain_income, ytrain))
print(log_income.score(xtest_income, ytest))

## 머신러닝_ Randomforest

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rdf_loan = RandomForestClassifier(max_depth=10).fit(xtrain_loan, ytrain)
rdf_credit = RandomForestClassifier(max_depth=10).fit(xtrain_credit, ytrain)
rdf_income = RandomForestClassifier(max_depth=10).fit(xtrain_income, ytrain)

In [ ]:
print(rdf_loan.score(xtrain_loan, ytrain))
print(rdf_loan.score(xtest_loan, ytest))
print('-'*20)
print(rdf_credit.score(xtrain_credit, ytrain))
print(rdf_credit.score(xtest_credit, ytest))
print('-'*20)
print(rdf_income.score(xtrain_income, ytrain))
print(rdf_income.score(xtest_income, ytest))

## 딥러닝_ tensorflow

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers

In [ ]:
model_loan = tf.keras.Sequential()
model_loan.add(layers.Input(shape=xtrain_loan.shape[1]))
model_loan.add(layers.Dense(190, activation='relu'))
model_loan.add(layers.Dense(19, activation='relu'))
model_loan.add(layers.Dense(1, activation='sigmoid'))

model_loan.compile(optimizer='Adam', loss='binary_crossentropy', metrics=['binary_accuracy'])
history_loan = model_loan.fit(xtrain_loan, ytrain, epochs=100)

In [ ]:
model_credit = tf.keras.Sequential()
model_credit.add(layers.Input(shape=xtrain_credit.shape[1]))
model_credit.add(layers.Dense(60, activation='relu'))
model_credit.add(layers.Dense(6, activation='relu'))
model_credit.add(layers.Dense(1, activation='sigmoid'))

model_credit.compile(optimizer='Adam', loss='binary_crossentropy', metrics=['binary_accuracy'])
history_credit = model_credit.fit(xtrain_credit, ytrain, epochs=100)

In [ ]:
model_income = tf.keras.Sequential()
model_income.add(layers.Input(shape=xtrain_income.shape[1]))
model_income.add(layers.Dense(170, activation='relu'))
model_income.add(layers.Dense(17, activation='relu'))
model_income.add(layers.Dense(1, activation='sigmoid'))

model_income.compile(optimizer='Adam', loss='binary_crossentropy', metrics=['binary_accuracy'])
history_income = model_income.fit(xtrain_income, ytrain, epochs=100)

In [ ]:
prediction_loan = model_loan.predict(xtest_loan)
prediction_credit = model_credit.predict(xtest_credit)
prediction_income = model_income.predict(xtest_income)
pred_loan = pd.DataFrame(prediction_loan, index= ytest.index)
pred_credit =pd.DataFrame(prediction_credit, index= ytest.index)
pred_income =pd.DataFrame(prediction_income, index= ytest.index)
predsum = pd.concat([pred_loan, pred_credit, pred_income], axis =1)
predsum.columns = ['loan', 'credit', 'income']
predsum

In [ ]:
finx_train, finx_test, finy_train, finy_test = train_test_split(predsum, ytest, test_size=0.2, stratify = ytest)

In [ ]:
model_total = tf.keras.Sequential()
model_total.add(layers.Input(shape=finx_train.shape[1]))
model_total.add(layers.Dense(30, activation='relu'))
model_total.add(layers.Dense(3, activation='relu'))
model_total.add(layers.Dense(1, activation='sigmoid'))

model_total.compile(optimizer='Adam', loss='binary_crossentropy', metrics=['binary_accuracy'])
history_total = model_total.fit(finx_train, finy_train, epochs=100)

In [ ]:
fin_pred = model_total.predict(finx_test)
fin_pred = fin_pred.reshape(3541,)
fin_pred

In [ ]:
result=[]
for i in range(len(fin_pred)):
  if fin_pred[i]>=0.5:
    result += [1]
  elif fin_pred[i]<0.5:
    result += [0]


In [ ]:
from sklearn.metrics import confusion_matrix
pd.DataFrame(confusion_matrix(finy_test, result), index=['Real_0', 'Real_1'], columns=['Pred_0', 'Pred_1'])